# <center> Final Project Code: Grocery Sales Prediction </center>
## <center> Big Data Analytics </center>
## <center> By: Urvi Mistry | Purva Kedari | Trisha Chakraborty </center>

# Data

The dataset we are using for Grocery Sales Prediction was retreived from Kaggle.
This dataset is a part of the Kaggle Competition named, Corporación Favorita Grocery Sales Forecasting. The goal of this project is to predict Sales based on Day and Month as well as Item number and Month.

Link to the Kaggle: https://www.kaggle.com/c/favorita-grocery-sales-forecasting/data


# Importing Libraries

In [ ]:
# Load the packages needed for this part
# create spark and sparkcontext objects
from pyspark.sql import SparkSession
import numpy as np

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

import pyspark
from pyspark.ml import feature, regression, Pipeline, evaluation

from pyspark.sql import functions as fn, Row
from pyspark.sql.functions import isnan, when, count
from pyspark import sql
from pyspark.sql.window import Window
import pyspark.sql.functions as func

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

from pyspark.sql.types import IntegerType
import pylab

# Importing CSV Files

In [ ]:
#Importing csv files
test_df = spark.read.csv("test.csv", header=True).limit(120000)
train_df = spark.read.csv("train.csv", header=True)
items_df = spark.read.csv("items.csv", header=True)
stores_df = spark.read.csv("stores.csv", header=True)
holidays_df = spark.read.csv("holidays_events.csv", header=True)
transactions_df = spark.read.csv("transactions.csv",header=True)

# Preprocessing

In [ ]:
#Renaming some columns and dropping some unnecessary columns
stores_df = stores_df.withColumnRenamed("type","store_type")
holidays_df = holidays_df.withColumnRenamed("type","holiday_type")
holidays_df = holidays_df.drop('description','transferred')
train_df = train_df.drop('id')
test_df = test_df.drop('id')

In [ ]:
#As all the numerical columns were wrongly converted to String from csv files. Thus, converting those columns back to integer
train_df = train_df.withColumn("store_nbr", train_df["store_nbr"].cast(IntegerType()))
train_df = train_df.withColumn("item_nbr", train_df["item_nbr"].cast(IntegerType()))
train_df = train_df.withColumn("unit_sales", train_df["unit_sales"].cast(IntegerType()))
test_df = test_df.withColumn("store_nbr", test_df["store_nbr"].cast(IntegerType()))
test_df = test_df.withColumn("item_nbr", test_df["item_nbr"].cast(IntegerType()))
items_df = items_df.withColumn("item_nbr",items_df["item_nbr"].cast(IntegerType()))
items_df = items_df.withColumn("class",items_df["class"].cast(IntegerType()))
items_df = items_df.withColumn("perishable",items_df["perishable"].cast(IntegerType()))
stores_df = stores_df.withColumn("store_nbr",stores_df["store_nbr"].cast(IntegerType()))
stores_df = stores_df.withColumn("cluster",stores_df["cluster"].cast(IntegerType()))
transactions_df = transactions_df.withColumn("store_nbr",transactions_df["store_nbr"].cast(IntegerType()))
transactions_df = transactions_df.withColumn("transactions",transactions_df["transactions"].cast(IntegerType()))

# Sampling data

In [ ]:
train_df = train_df.sample(False, 0.001,0)

In [ ]:
test_df = test_df.sample(False, 0.1,0)

# Joining CSV Files

In [ ]:
#joining the dataframe into train_df
train_df = train_df.join(stores_df, on = 'store_nbr', how = 'left')
train_df = train_df.join(items_df, on = 'item_nbr', how = 'left')
train_df = train_df.join(holidays_df, on = 'date', how = 'left')
train_df = train_df.join(transactions_df, on = ['store_nbr','date'], how = 'left')

In [ ]:
#joining the dataframes into test_df
test_df = test_df.join(stores_df, on = 'store_nbr', how = 'left')
test_df = test_df.join(items_df, on = 'item_nbr', how = 'left')
test_df = test_df.join(holidays_df, on = 'date', how = 'left')
test_df = test_df.join(transactions_df, on = ['store_nbr','date'], how = 'left')

# Removing NULL values

Joining the csv files generates NULL values. For example, when holidays.csv was joined with train.csv there were NULL values generated, as train.csv contained all the dates and not all those dates were a holiday. 

Thus, we replaced these NULL values with some value. For example, dates which didn't have a holiday were replaced with NO Holiday.

In [ ]:
#Filling null values with some value
train_df = train_df.fillna({'onpromotion':'False','holiday_type':'No Holiday','locale':'Not Applicable',\
                            'locale_name':'Not Applicable', 'transactions':1884})

In [ ]:
test_df = test_df.fillna({'onpromotion':'False','holiday_type':'No Holiday','locale':'Not Applicable',\
                            'locale_name':'Not Applicable', 'transactions':1884})

# Removing negative values

Dataset had some errored values where the unit_sales column contained values less than 0 which was definitely errored. As these errored rows were very few (about 3), we decided to drop them from our dataset.

In [ ]:
train_df = (train_df.filter(train_df.unit_sales > 0))

In [ ]:
test_df = (train_df.filter(train_df.unit_sales > 0))

# Split date

Splitting the Date column into Day, Month and Year for performing Monthly analysis.

In [ ]:
split_date = pyspark.sql.functions.split(train_df['date'], '-')     
train_df= train_df.withColumn('Year', split_date.getItem(0))
train_df= train_df.withColumn('Month', split_date.getItem(1))
train_df= train_df.withColumn('Day', split_date.getItem(2))
train_df = train_df.drop('date')

In [ ]:
split_date = pyspark.sql.functions.split(test_df['date'], '-')     
test_df= test_df.withColumn('Year', split_date.getItem(0))
test_df= test_df.withColumn('Month', split_date.getItem(1))
test_df= test_df.withColumn('Day', split_date.getItem(2))
test_df = test_df.drop('date')

In [ ]:
test_df = test_df.withColumn("Day", test_df["Day"].cast(IntegerType()))
test_df = test_df.withColumn("Month", test_df["Month"].cast(IntegerType()))
test_df = test_df.withColumn("Year", test_df["Year"].cast(IntegerType()))
train_df = train_df.withColumn("Day", train_df["Day"].cast(IntegerType()))
train_df = train_df.withColumn("Month", train_df["Month"].cast(IntegerType()))
train_df = train_df.withColumn("Year", train_df["Year"].cast(IntegerType()))

# Onehotencoded Dummy Variables

The dataset contains a lot of categorical columns which are:
* onpromotion
* state
* store_type
* family
* city
* holiday_type
* locale
* locale_name

As all the above mentioned columns were valuable for unit sales prediction, therefore we converted them into Dummy variables using StringIndexer and OneHotEncoder.

In [ ]:
#Copying the train data into another variable
train_df1 = train_df

In [ ]:
#Dummy variable creation for train data
inputcolumns = ["onpromotion","state","store_type","family","city","holiday_type","locale","locale_name"]
indexer = [StringIndexer(stringOrderType = 'alphabetAsc', inputCol = col, outputCol = "{0}_index".format(col)) for col in inputcolumns]
encoder = [OneHotEncoder(inputCol = idx.getOutputCol(), outputCol = "{0}_feat".format(idx.getOutputCol()),dropLast = False) for idx in indexer]
assembler = VectorAssembler(inputCols=['onpromotion_index_feat','state_index_feat','store_type_index_feat'\
                                      ,'family_index_feat','city_index_feat','holiday_type_index_feat','locale_index_feat','locale_name_index_feat'\
                                      ,'store_nbr', 'item_nbr','cluster','class','perishable','Month','Day','transactions']\
                            ,outputCol="features")
pipeline = Pipeline(stages=indexer + encoder + [assembler])
model = pipeline.fit(train_df1)
transformed = model.transform(train_df1)

In [ ]:
#Dummy variable creation for test data
inputcolumns = ["onpromotion","state","store_type","family","city","holiday_type","locale","locale_name"]
testindexer = [StringIndexer(stringOrderType = 'alphabetAsc', inputCol = col, outputCol = "{0}_index".format(col)) for col in inputcolumns]
testencoder = [OneHotEncoder(inputCol = idx.getOutputCol(), outputCol = "{0}_feat".format(idx.getOutputCol()),dropLast = False) for idx in indexer]
testassembler = VectorAssembler(inputCols=['onpromotion_index_feat','state_index_feat','store_type_index_feat'\
                                      ,'family_index_feat','city_index_feat','holiday_type_index_feat','locale_index_feat','locale_name_index_feat'\
                                      ,'store_nbr', 'item_nbr','cluster','class','perishable','Month','Day','transactions']\
                            ,outputCol="features")
testpipeline = Pipeline(stages=indexer + encoder + [assembler])
testmodel = pipeline.fit(test_df)
testtransformed = model.transform(test_df)

Dropping categorical columns and indexed columns thereby keeping only the numerical columns and dummy columns

In [ ]:
transformed = transformed.drop('onpromotion','onpromotion_index','state','state_index','city','city_index',\
                               'store_type','store_type_index','family','family_index','holiday_type','holiday_type_index',\
                               'locale','locale_index','locale_name','locale_name_index')

In [ ]:
testtransformed = testtransformed.drop('onpromotion','onpromotion_index','state','state_index','city','city_index',\
                               'store_type','store_type_index','family','family_index','holiday_type','holiday_type_index',\
                               'locale','locale_index','locale_name','locale_name_index')

In [ ]:
# Sample Code for performing PCA on the dataset. As the scree plot for PCA was unclear and almost even. We decided to not perform PCA on the dataset.

In [ ]:
# # assembler2 = feature.VectorAssembler(inputCols = ['store_nbr','item_nbr','cluster','class','perishable'\
# #                                                  ,'transactions','Month','Day','features'],outputCol='pcafeatures')
# std_scaled2 = feature.StandardScaler(inputCol='features', outputCol='standardizedFeatures')
# pca3 = feature.PCA(k=17, inputCol='standardizedFeatures', outputCol='pc')
# dpipe_pca = Pipeline(stages = [std_scaled2, pca3]).fit(training_df)
# dtrain = dpipe_pca.transform(training_df)

In [ ]:
# pca_model = dpipe_pca.stages[-1]
# pc1 = np.absolute(pca_model.pc.toArray()[:, 0]).tolist()
# pc2 =np.absolute(pca_model.pc.toArray()[:, 1]).tolist()

In [ ]:
# explainedVariance = dpipe_pca.stages[-1].explainedVariance
# explainedVariance

In [ ]:
# import numpy as np
# y = explainedVariance.toArray().tolist()
# y = np.sort(np.cumsum(y))/np.sum(y)
# plt.plot(y)
# plt.xlabel('number of components')
# plt.ylabel('Explained variance');

# Model 1: Linear Regression

Peforming Linear Regression on the dataset containing all the features

In [ ]:
#Since we have a spearate test dataset we split the train into training and validation
lineartrainingdf, linearvalidationdf = transformed.randomSplit([0.7, 0.3], seed=0)

In [ ]:
#Retrieving columns
pcacols = [x for x in lineartrainingdf.columns]
pcacols.pop(2)
pcacols.pop(6)

In [ ]:
lrcols = pcacols

In [ ]:
# train_df.select(fn.max("unit_sales")).show()

In [ ]:
# train_df.select(fn.min("unit_sales")).show()

In [ ]:
#Defining Model 1 pipeline and fitting it on training data
lrmodel1 = Pipeline(stages=[
#     feature.VectorAssembler(inputCols = pcacols, outputCol = 'features'),
    regression.LinearRegression(featuresCol='features', labelCol='unit_sales')  
]).fit(lineartrainingdf)


In [ ]:
a = lrmodel1.transform(linearvalidationdf)

In [ ]:
rmse = fn.sqrt(fn.mean((fn.col('unit_sales') - fn.col('prediction'))**2)).alias('rmse')

In [ ]:
#Calculating RMSE of Model1
lrmodel1.transform(linearvalidationdf).select(rmse).show()

In [ ]:
#Retreiving Coefficients
coef = lrmodel1.stages[-1].coefficients.toArray()
coef

In [ ]:
#Retriving the labels of the OneHotEncoded columns to better represent the coefficients
sales_col_list = ['item_nbr','store_nbr','cluster','class','perishable','Month','Day']

l1 = model.stages[0].labels
for i in l1:
    label1 = i.replace(' ','_')
    sales_col_list.append('onpromotion_'+ label1)
    
l2 = model.stages[1].labels
for i in l2:
    label2 = i.replace(' ','_')
    sales_col_list.append('state_'+ label2)
    
l3 = model.stages[2].labels
for i in l3:
    label3 = i.replace(' ','_')
    sales_col_list.append('store_type_'+ label3)
    
l4 = model.stages[3].labels
for i in l4:
    label4 = i.replace(' ','_')
    sales_col_list.append('family_'+ label4)
    
l5 = model.stages[4].labels
for i in l5:
    label5 = i.replace(' ','_')
    sales_col_list.append('holiday_type_'+ label5)
    
l6 = model.stages[5].labels
for i in l6:
    label6= i.replace(' ','_')
    sales_col_list.append('locale_'+ label6)
    
l7 = model.stages[6].labels
for i in l7:
    label7= i.replace(' ','_')
    sales_col_list.append('locale_name_'+ label7)
    
l8 = model.stages[7].labels
for i in l8:
    label8 = i.replace(' ','_')
    sales_col_list.append('city_'+ label8)


In [ ]:
#Feature Importance
feature_importance = pd.DataFrame(list(zip(sales_col_list, coef)),\
                                 columns = ['feature', 'Coefficient']).sort_values('Coefficient', ascending=False)

In [ ]:
feature_importance = feature_importance.loc[feature_importance['Coefficient']>0.05]

In [ ]:
feature_importance.head(10)

# Model 2: Random Forest

Performing Random Forest on the dataset containing all the features.

In [ ]:
train_df2 = train_df
transformed.dtypes

In [ ]:
rftraining_df, rfvalidation_df = transformed.randomSplit([0.7, 0.3], seed=0)

In [ ]:
#Fitting Model 2 pipeline
RF_pipe = Pipeline(stages = [regression.RandomForestRegressor(featuresCol='features', labelCol='unit_sales')]).fit(rftraining_df)

In [ ]:
evaluator = evaluation.RegressionEvaluator(labelCol='unit_sales', metricName='rmse')

In [ ]:
#Evaluating RMSE of Model 2
rmseRF=evaluator.evaluate(RF_pipe.transform(rfvalidation_df))

In [ ]:
print(rmseRF)

In [ ]:
#Removing Year column
pcacols.pop(15)

In [ ]:
#Feature Importance
imp = RF_pipe.stages[-1].featureImportances.toArray()
feature_importance = pd.DataFrame(list(zip(pcacols, imp)),\
                                 columns = ['feature', 'importance']).sort_values('importance', ascending=False)

In [ ]:
feature_importance.head(10)

# Plot

Plotting the Actual Sales vs Predicted Sales plot

In [ ]:
a.show()

In [ ]:
x1 = a.groupBy('Month').agg(fn.sum('unit_sales').alias('Unit_Sales'),fn.sum('prediction').alias('Predicted_Sales')).toPandas()

In [ ]:
sns.lineplot(x="Month",y="Unit_Sales", data = x1)
sns.lineplot(x="Month",y="Predicted_Sales", data = x1)
plt.legend(labels=['Actual Sales','Predicted Sales'])
plt.ylabel("Sum of Sales")

# Model 3: Linear Regression with Important Features

Performing Linear Regression the important features which were retrived from the feature importance method

In [ ]:
transformed.dtypes

In [ ]:
moderate = transformed.select("item_nbr","Day","Month","perishable","class","family_index_feat","unit_sales")
moderate = moderate.groupby("item_nbr","Month","Day","perishable","class","family_index_feat").agg(fn.sum("unit_sales"))
lineartrainingdf2, linearvalidationdf2 = moderate.randomSplit([0.7, 0.3], seed=0)

In [ ]:
lineartrainingdf2.dtypes

In [ ]:
lrmodcols = ['item_nbr','Month','Day','perishable','class','family_index_feat']

In [ ]:
#Fitting Model3 pipeline
lrmodel2 = Pipeline(stages=[
    feature.VectorAssembler(inputCols = lrmodcols, outputCol = 'features'),
    regression.LinearRegression(featuresCol='features', labelCol='sum(unit_sales)')  
]).fit(lineartrainingdf2)


In [ ]:
lrmodel2.transform(linearvalidationdf2)

In [ ]:
rmse2 = fn.sqrt(fn.mean((fn.col('sum(unit_sales)') - fn.col('prediction'))**2)).alias('rmse')

In [ ]:
#Evaluating RMSE of Model 3
lrmodel2.transform(linearvalidationdf2).select(rmse2).show()

# Model 4: Random Forest with Important Features 

Performing Random Forest on Important Features retrieved from Feature Importance method.

In [ ]:
moderatef = transformed.select("item_nbr","Day","Month","perishable","unit_sales")
moderatef = moderatef.groupby("item_nbr","Month","Day","perishable").agg(fn.sum("unit_sales"))
randomtrainingdf2, randomvalidationdf2 = moderatef.randomSplit([0.7, 0.3], seed=0)

In [ ]:
lrmodcols.pop(4)
lrmodcols.pop(4)
# lrmodcols.pop(3)

In [ ]:
lrmodcols

In [ ]:
RF_modpipe = Pipeline(stages = [feature.VectorAssembler(inputCols = lrmodcols, outputCol = 'features'),\
                                regression.RandomForestRegressor(featuresCol='features', labelCol='sum(unit_sales)')]).fit(randomtrainingdf2)

In [ ]:
evaluator = evaluation.RegressionEvaluator(labelCol='sum(unit_sales)', metricName='rmse')

In [ ]:
#Evaluating RMSE of Model 4
rmseRF=evaluator.evaluate(RF_modpipe.transform(randomvalidationdf2))

In [ ]:
print(rmseRF)

In [ ]:
final_model = RF_modpipe

# Prediction

Since Random Forest with Important features had less RMSE than another other Model. We are performing Prediction based on the Model 4 as the final model.

We have created 2 functions below for predicting sales:
1) Predict Unit Sales by Month and Day
2) Predict Unit Sales by Month and Item Number

In [ ]:
final_model.transform(randomvalidationdf2).groupby('Month','Day').agg(fn.avg('sum(unit_sales)')).show()

In [ ]:
#Function for Predicting Sales by Month and Day
def PredictSalesDay(M, D):
    abc = randomtrainingdf2.filter(fn.col('Month') == M)
    abc = abc.filter(fn.col('Day') == D)
    data = final_model.transform(abc)
    sales = data.select(fn.avg('prediction')).alias("Predicted Sales").show()

In [ ]:
PredictSalesDay(1,25)

In [ ]:
# testmod = testtransformed.select("item_nbr","Day","Month","perishable","class","family_index_feat","unit_sales")
# testmod = testmod.groupby("item_nbr","Month","Day","perishable","class","family_index_feat").agg(fn.sum("unit_sales"))
# randomtrainingdf2, randomvalidationdf2 = moderatef.randomSplit([0.7, 0.3], seed=0)

In [ ]:
# final_model.transform(randomvalidationdf2).groupby('Month','item_nbr').agg(fn.sum('sum(unit_sales)')).show()

In [ ]:
# testmodel = Pipeline(stages=[
#     feature.VectorAssembler(inputCols = pcacols, outputCol = 'features'),
#     regression.RandomForestRegressor(featuresCol='features', labelCol='sum(unit_sales)')  
# ]).fit(testtransformed)
final_model = RF_modpipe

In [ ]:
testmoderatef = testtransformed.select("item_nbr","Day","Month","perishable","class","family_index_feat")

In [ ]:
#Predicting Sales by Month and Item number
def PredictSales(M, I):
    abc = randomtrainingdf2.filter(fn.col('Month') == M)
    abc = abc.filter(fn.col('item_nbr') == I)
    data = final_model.transform(abc)
#     data = 
    sales = data.select(fn.sum('prediction')).alias("Predicted Sales").show()

In [ ]:
PredictSales(12,2130553)

# Suggestions 

As it was suggested by Professor during the Poster Presentation, we created a dataframe containing sales prediction for the provided item number, Month, Day and perishable.

In [ ]:
abcd = spark.createDataFrame([(623316,12,10,1)],['item_nbr','Month','Day','perishable'])

In [ ]:
data = final_model.transform(abcd)

In [ ]:
data.select('prediction').show()

In [ ]:
abcd.show()

In [ ]:
testtransformed.show()